# Compute Relative Power of EEG Bands

## Goals:
1. **Data Import**
    - Import the preprocessed data from npz and json files

2. **Feature Extraction**
    - Average the time series data for each epoch over the occipital channels
    - Calculate the Power Spectral Density (PSD) for each epoch.
    - Average PSD values across all epochs 
    - Get AUC across freq values for each frequency band
    
3. **Data Formatting**
    - Export the data as (1, 12, 1, ch_num, ch_num, ch_num, ch_num)
        - (file, stimuli, comfort score, gamma amplitude, theta amplitude, alpha amplitude, beta amplitude)

# Import Libraries

In [1]:
import json
import numpy as np
import pandas as pd
import scipy.signal as signal
import matplotlib.pyplot as plt

# Custom libraries
from Functions import processing

%matplotlib qt

# Import Epoched Data and Settings

In [2]:
# Load list of files to import
files = [  
    "sub-P002_ses-S001_task-T1_run-001_eeg"   
]

# Get unique subject IDs
subject_ids = [file.split('_')[0] for file in files]
unique_subject_ids = list(set(subject_ids))

# Preallocate variables to store EEG data and settings
eeg_epochs = [None] * len(files)
settings = [None] * len(files)

# Import data
for f, file in enumerate(files):
    # Import EEG data, since it is stored in a compressed numpy file (.npz) we need to use the np.load function 
    loaded_data = np.load(f"Data\\Masters_testing\\P002\\{file}.npz", allow_pickle=True)

    # Access the data for each stimulus
    eeg_epochs[f] = {stim_label: loaded_data[stim_label] for stim_label in loaded_data.files}

    # Import settings
    with open(f"Data\\Masters_testing\\P002\\{file}.json", "r") as file_object:
        settings[f] = json.load(file_object)

# Hardcoding for now
stim_comfort_dict = dict({'Contrast1Size1': 6, 'Contrast1Size2': 7, 'Contrast1Size3': 6.67, 'Contrast2Size1': 5.67, 'Contrast2Size2': 6.25, 'Contrast2Size3': 6.5, 'Contrast3Size1': 1.75, 'Contrast3Size2': 4.67, 'Contrast3Size3': 5.33, 'Contrast4Size1': 1.67, 'Contrast4Size2': 2, 'Contrast4Size3': 3.8})

# Compute PSD for each "On" Epoch

- Calculate AUC for every channel for 4 freq bands
    - Do not include the SSVEP stim freqs/harmonics in the frequency bands

In [3]:
# PSD settings
window_size = 10  # 10 = 0.1 Hz resolution, 5 = 0.2 Hz resolution, 2 = 0.5 Hz resolution

# Preallocate variables
eeg_f = [None] * len(files)
eeg_pxx = [None] * len(files)  

# Compute PSD for each file
for f in range(len(files)):
    eeg_f[f] = {}
    eeg_pxx[f] = {}

    for stim_label, epochs in eeg_epochs[f].items():  
        eeg_f[f][stim_label] = []
        eeg_pxx[f][stim_label] = []

        for epoch in epochs:  
            f_values, pxx_values = signal.welch(
                x=epoch,  
                fs=settings[f]["eeg_srate"],
                nperseg=window_size * settings[f]["eeg_srate"],
                noverlap=(window_size * settings[f]["eeg_srate"]) * 0.5,  
            )
            eeg_f[f][stim_label].append(f_values)
            eeg_pxx[f][stim_label].append(pxx_values)

        eeg_f[f][stim_label] = np.array(eeg_f[f][stim_label])  # (num_epochs, num_frequencies)
        eeg_pxx[f][stim_label] = np.array(eeg_pxx[f][stim_label])  # (num_epochs, num_frequencies)

#print shape of PSD for each file and stim
#for f in range(len(files)):
#    print(f"File {f}:")
#    for stim_label in eeg_pxx[f]:
#        print(f"\tStimulus {stim_label}: {eeg_pxx[f][stim_label].shape}")
# Define Frequency Bands
eeg_bands = {"delta": [1, 4], "theta": [4, 8], "alpha1": [8, 10], "alpha2": [11, 13], "beta1": [13, 20], "beta2": [21, 30]}

# Preallocate AUC array: (num_files, num_stimuli, num_channels, num_bands)
auc = np.zeros((len(files), len(eeg_pxx[0]), len(settings[f]['new_ch_names']), len(settings[f]['new_ch_names']), len(settings[f]['new_ch_names']), len(settings[f]['new_ch_names'])))  # Adjusted shape

# Iterate over files and stimuli
for f in range(len(files)):
    for stim_idx, stim_label in enumerate(eeg_pxx[f]):
        for band_idx, (band_name, band_range) in enumerate(eeg_bands.items()):
            # Fix fmask indexing for the frequency range
            fmask = (eeg_f[f][stim_label][0, :] >= band_range[0]) & (eeg_f[f][stim_label][0, :] <= band_range[1])

            # Average across epochs first: (num_channels, num_frequencies)
            avg_psd = np.mean(eeg_pxx[f][stim_label][:, :, fmask], axis=0)  # (num_channels, num_frequencies)

            # Print the shape of the average PSD for each band
            #print(f"File {f}, Stimulus {stim_label}, Band {band_name}: {avg_psd.shape}")

            # Integrate the PSD across the frequency range for each band
            if band_name == 'delta':
                delta = np.trapz(avg_psd, x=eeg_f[f][stim_label][0, fmask], axis=1)
                auc[f, stim_idx] = delta
            elif band_name == 'theta':
                theta = np.trapz(avg_psd, x=eeg_f[f][stim_label][0, fmask], axis=1)
                auc[f, stim_idx, :] = theta
            elif band_name == 'alpha1':
                alpha1 = np.trapz(avg_psd, x=eeg_f[f][stim_label][0, fmask], axis=1)
                # Print the shape for alpha1
                print(f"File {f}, Stimulus {stim_label}, Band {band_name}: {alpha1.shape}")
            elif band_name == 'alpha2':
                alpha2 = np.trapz(avg_psd, x=eeg_f[f][stim_label][0, fmask], axis=1)
                # Print the shape for alpha2
                print(f"File {f}, Stimulus {stim_label}, Band {band_name}: {alpha2.shape}")
                full_alpha = alpha1 + alpha2
                print(f"File {f}, Stimulus {stim_label}, Band full_alpha: {full_alpha.shape}")
                auc[f, stim_idx, :, :] = full_alpha
            elif band_name == 'beta1':
                beta1 = np.trapz(avg_psd, x=eeg_f[f][stim_label][0, fmask], axis=1)
            elif band_name == 'beta2':
                beta2 = np.trapz(avg_psd, x=eeg_f[f][stim_label][0, fmask], axis=1)
                full_beta = beta1 + beta2
                auc[f, stim_idx, :, :, :] = full_beta

# Print shape of the final AUC array
print(f"AUC: {auc.shape}")


File 0, Stimulus Contrast1Size1, Band alpha1: (13,)
File 0, Stimulus Contrast1Size1, Band alpha2: (13,)
File 0, Stimulus Contrast1Size1, Band full_alpha: (13,)
File 0, Stimulus Contrast1Size2, Band alpha1: (13,)
File 0, Stimulus Contrast1Size2, Band alpha2: (13,)
File 0, Stimulus Contrast1Size2, Band full_alpha: (13,)
File 0, Stimulus Contrast1Size3, Band alpha1: (13,)
File 0, Stimulus Contrast1Size3, Band alpha2: (13,)
File 0, Stimulus Contrast1Size3, Band full_alpha: (13,)
File 0, Stimulus Contrast2Size1, Band alpha1: (13,)
File 0, Stimulus Contrast2Size1, Band alpha2: (13,)
File 0, Stimulus Contrast2Size1, Band full_alpha: (13,)
File 0, Stimulus Contrast2Size2, Band alpha1: (13,)
File 0, Stimulus Contrast2Size2, Band alpha2: (13,)
File 0, Stimulus Contrast2Size2, Band full_alpha: (13,)
File 0, Stimulus Contrast2Size3, Band alpha1: (13,)
File 0, Stimulus Contrast2Size3, Band alpha2: (13,)
File 0, Stimulus Contrast2Size3, Band full_alpha: (13,)
File 0, Stimulus Contrast3Size1, Band al

# Combine stimuli, comfort scores, and PSD (per freq band)

Shape will be (1, 12, 12, n_chan, n_chan, n_chan, n_chan)
- for each stim it will be arr[f,stim] = (1, n_chan, n_chan, n_chan, n_chan)

In [4]:
# Preallocate array with the given shape
ml_input = np.zeros((
    len(files),                        # Number of files
    len(eeg_pxx[0]),                   # Number of stimuli
    1,                                 # Number of comfort scores per stimulus
    len(settings[f]['new_ch_names']),  # Number of channels, delta band
    len(settings[f]['new_ch_names']),  # Number of channels, theta band
    len(settings[f]['new_ch_names']),  # Number of channels, alpha band
    len(settings[f]['new_ch_names'])   # Number of channels, beta band
))  


# Fill in the array
for f in range(len(files)):
    for stim_idx, stim_label in enumerate(eeg_pxx[f]):
        for comfort_idx, (c_stim_label, comfort_score) in enumerate(stim_comfort_dict.items()):
            if stim_label == c_stim_label:  # Ensure matching stimulus names
                for band_idx, (band_name, band_range) in enumerate(eeg_bands.items()):
                    ml_input[f, stim_idx, 0, :, :, :, :] = auc[f, stim_idx, :, :, :, :]

                    #print(f"File {f}, Stimulus {stim_label}, Comfort Score {comfort_score}, Band {band_name}: {ml_input[f, stim_idx, 0].shape}")

# Export ML Input to npz

In [5]:
save_data = False

if save_data:

    # Create dictionary to save each stimulus separately
    save_dict = {}
    for stim_idx, stim_label in enumerate(eeg_pxx[0]):
        save_dict[stim_label] = ml_input[:, stim_idx]

    # Save using np.savez to preserve structure
    np.savez(f"Data\\Masters_testing\\P002\\ml_input.npz", **save_dict)

    print(f"Saved organized epochs to Data\\ml_input.npz")